# Perspective program:

In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from IPython.display import display
import math
import numpy as np

In [2]:
obj = [
    [
        [ 1,    0, 1], 
        [ 0.5,  0.8660254037844386, 1], 
        [-0.5,  0.8660254037844387, 1], 
        [-1,  0, 1], 
        [-0.5, -0.8660254037844385, 1], 
        [ 0.5, -0.866025403784439, 1]
    ],
    [         
        [ 0.5, -0.866025403784439, -1],
        [-0.5, -0.8660254037844385, -1],
        [-1,  0, -1],
        [-0.5,  0.8660254037844387, -1], 
        [ 0.5,  0.8660254037844386, -1], 
        [ 1,    0, -1]
    ],
    [
        [ 0.5,  0.8660254037844387,  1],
        [ 0.5,  0.8660254037844386, -1], 
        [ 1, 0, -1],
        [ 1, 0,  1]
    ],
    [
        [ 1, 0,  1],
        [ 1, 0, -1],
        [ 0.5, -0.866025403784439, -1],
        [ 0.5, -0.866025403784439,  1]
    ],
    [
        [ 0.5, -0.866025403784439,   1],
        [ 0.5, -0.866025403784439,  -1],
        [-0.5, -0.8660254037844385, -1],
        [-0.5, -0.8660254037844385,  1]
    ],
    [
        [-0.5, -0.8660254037844385,  1],
        [-0.5, -0.8660254037844385, -1],
        [-1,  0, -1],
        [-1,  0,  1]
    ]]
perspective = [[0, 0, -1]]
colors = np.random.rand(len(obj), 3)

In [3]:
class spinning_object:

    def __init__(self):
        self.object = []
        self.colors = []
        self.perspective = []
        self.fig, self.ax = plt.subplots()
    
    def set_object(self, object):
        self.object = object

    def set_colors(self, colors):
        self.colors = colors

    def set_perspective(self, perspective):
        self.perspective = perspective

    def _general_rotation(self, theta):
    
        thetaRad = math.radians(theta)
        rotationYMatrix = [[ math.cos(thetaRad), 0, math.sin(thetaRad)],
                           [ 0,                  1, 0                 ],
                           [-math.sin(thetaRad), 0, math.cos(thetaRad)]]
    
        rotationXMatrix = [[1, 0,                   0                 ],
                           [0, math.cos(thetaRad), -math.sin(thetaRad)],
                           [0, math.sin(thetaRad),  math.cos(thetaRad)]]
        
        rotationZMatrix = [[math.cos(thetaRad), -math.sin(thetaRad), 0],
                           [math.sin(thetaRad),  math.cos(thetaRad), 0],
                           [0,                   0,                  1]]
        
        newObject = []

        for square in self.object:
            newSquare = []

            for point in square:
                
                result = np.dot(point, rotationXMatrix)
                result = np.dot(result, rotationYMatrix)
                result = np.dot(result, rotationZMatrix)
                newSquare.append(result)

            newObject.append(newSquare)
        
        return newObject

    def spinObject(self, theta: int, interval: int):
        anim = FuncAnimation(self.fig, self.animate, frames = theta, init_func=self.initAnim, interval=interval, blit=True)
        video = anim.to_html5_video()
        html = HTML(video)  
        display(html)
        plt.close()

    def animate(self, frame):
        self.ax.clear()
        self.ax.set_xlim(-5, 5)
        self.ax.set_ylim(-5, 5)
        self.ax.set_aspect("equal")
        new_object = self._general_rotation(frame)
        patches_list = self.printImage(new_object)
        return patches_list

    def initAnim(self):
        self.ax.set_xlim(-5, 5)
        self.ax.set_ylim(-5, 5)
        return []

    def calculate_average_distance(self, point):
        distances = [np.linalg.norm(np.array(point) - np.array(self.perspective)) for point in point]
        return np.mean(distances)

    def sort_planes_by_distance(self, new_object: list):
        distances = [self.calculate_average_distance(plane) for plane in new_object]
        planes_with_distances = list(zip(new_object, self.colors, distances))
        planes_sorted = sorted(planes_with_distances, key=lambda x: x[2], reverse=True)
        sorted_planes = [plane for plane, _, _ in planes_sorted]
        sorted_colors = [color for _, color, _ in planes_sorted]
        return sorted_planes, sorted_colors

    def printImage(self, new_object: list):

        patches_list = []
        planes, colors = self.sort_planes_by_distance(new_object)

        for i, square in enumerate(planes):
            newPoints = [(point[0], point[1]) for point in square]
            square = patches.Polygon(newPoints, closed=True, fill=True, color=colors[i])
            self.ax.add_patch(square)
            patches_list.append(square)

        return patches_list


In [4]:
classO = spinning_object()
classO.set_object(obj)
classO.set_colors(colors)
classO.set_perspective(perspective)

classO.spinObject(360, 20)